# Install dependencies

In [37]:
%pip install -r requirements.txt -q

Note: you may need to restart the kernel to use updated packages.


# imports

In [38]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import random
import os
from datetime import datetime, timedelta
import tradermade
import yfinance as yf

# Constants

In [39]:
TRADERMADE_API_KEY = os.getenv("TRADERMADE_API_KEY")
CURRENCY = "XAUUSD"
REQUIRED_COLUMNS = ['Open', 'High', 'Low', 'Close']
TICKER = "SPY"

END_DATE = datetime.now()
START_DATE = END_DATE - timedelta(days=365)
INTERVAL = "1d"

# Initialization

In [40]:
tradermade.set_rest_api_key(TRADERMADE_API_KEY)

# Get unstructured data (tick level)

## Get from yfinance (bars)

In [41]:
# df = yf.download(TICKER, start=START_DATE, end=END_DATE, interval=INTERVAL, auto_adjust=False, ignore_tz=True)

### Preprocessing data

#### Handle MultiIndex columns

In [42]:
# if isinstance(df.columns, pd.MultiIndex):
#     df = df.xs(TICKER, axis=1, level=1)
#     df.columns = [col.title() for col in df.columns]
# else:
#     df.columns = [col.title() for col in df.columns]

#### Convert required columns to numeric and drop rows with NaN

In [43]:
# df[REQUIRED_COLUMNS] = df[REQUIRED_COLUMNS].apply(pd.to_numeric, errors='coerce')
# df = df.dropna(subset=REQUIRED_COLUMNS)
# df.index = df.index.tz_localize(None)

## Mock data

In [44]:
def generate_random_trade_times(n_trades, start_time, end_time):
    start_ts = pd.to_datetime(start_time)
    end_ts = pd.to_datetime(end_time)

    total_seconds = int((end_ts - start_ts).total_seconds())

    # Generate n_trades random seconds within the time range
    random_seconds = sorted(random.sample(range(total_seconds), n_trades))
    
    return [start_ts + timedelta(seconds=s) for s in random_seconds]


def generate_mock_trades(num_trades=1000, start_price=100.0, start_time="2025-07-14 09:30:00", end_time="2025-07-14 16:00:00"):
    timestamps = generate_random_trade_times(
        n_trades=1000,
        start_time=start_time,
        end_time=end_time
    )

    prices = [start_price]
    for _ in range(1, num_trades):
        # Simulate small price changes
        change = np.random.normal(loc=0, scale=0.05)
        prices.append(round(prices[-1] + change, 2))

    volumes = np.random.randint(1, 1000, size=num_trades)

    df = pd.DataFrame({
        'timestamp': timestamps,
        'price': prices,
        'volume': volumes
    })
    return df

df = generate_mock_trades()
print(df)

              timestamp   price  volume
0   2025-07-14 09:30:38  100.00     847
1   2025-07-14 09:31:00  100.02     979
2   2025-07-14 09:31:03   99.99      39
3   2025-07-14 09:31:42   99.88     871
4   2025-07-14 09:32:05   99.89     507
..                  ...     ...     ...
995 2025-07-14 15:57:36   98.95     364
996 2025-07-14 15:58:51   98.92     143
997 2025-07-14 15:59:00   98.86     615
998 2025-07-14 15:59:06   98.90     623
999 2025-07-14 15:59:47   98.89     320

[1000 rows x 3 columns]


# Convert unstructured data to bars

## Time bars

## Tick bars

In [47]:
def generate_tick_bars(unstructured_data, tick_rate=5):
    return unstructured_data[::tick_rate]

generate_tick_bars(df)

,timestamp,price,volume
0,2025-07-14 09:30:38,100.00,847
5,2025-07-14 09:32:12,99.85,160
10,2025-07-14 09:33:38,99.93,166
15,2025-07-14 09:36:58,99.76,106
20,2025-07-14 09:39:21,99.76,74
...,...,...,...
975,2025-07-14 15:51:30,98.89,697
980,2025-07-14 15:53:08,99.01,896
985,2025-07-14 15:54:47,99.11,917
990,2025-07-14 15:55:21,98.91,682


## Volume bars

## Dollar bars

# Plot bars

## Time bars

In [46]:
fig = go.Figure(data=[go.Candlestick(
    x=df.index,
    open=df['Open'],
    high=df['High'],
    low=df['Low'],
    close=df['Close'],
    name='SPY'
)])

fig.update_layout(
    title=f'SPY (S&P 500 ETF) Candlestick Chart - Last 60 Days',
    yaxis_title='Price (USD)',
    xaxis_title='Date',
    xaxis_rangeslider_visible=True, 
    template='seaborn', # other options: plotly, plotly_white, plotly_dark, simple_white
    showlegend=True,
)

KeyError: 'Open'

## Volume bars

In [ ]:
# Assume each row = 1 trade (approximation)
tick_size = 1000  # Create a bar every 1000 "ticks"
tick_bars = df.iloc[::tick_size]  # Slice every N rows

cumulative_volume = df['Volume'].cumsum()
volume_bar_size = 10000  # New bar every 10,000 shares traded

# Group by volume thresholds
volume_bars = df.groupby(cumulative_volume // volume_bar_size).agg({
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last',
    'Volume': 'sum'
})

import plotly.graph_objects as go

fig = go.Figure(data=[go.Candlestick(
    x=volume_bars.index,
    open=volume_bars['Open'],
    high=volume_bars['High'],
    low=volume_bars['Low'],
    close=volume_bars['Close']
)])

fig.update_layout(
    title="AAPL Volume Bars (10,000 shares/bar)",
    yaxis_title="Price",
    xaxis_title="Volume Bar Index",
    template="plotly_dark"
)
fig.show()